# Exponential Thick Disc

Density profile : 


\begin{equation}
\label{eq:df_disc}
\rho(R, z) = \rho_{d, 0} \rho_R (R) \rho_z (z)
\end{equation}

where 

$$ \rho_R (R) = e^{\alpha R}$$

and $$ \rho_z (z) = \cosh^{-2}{\left(\frac{\beta z}{2} \right)}$$

Result is : 

\begin{equation}
\label{eq:sol_disc}
\Phi(R, z) = - 2\pi G \alpha \rho_{d, 0} \int_{0}^{\infty} dk J_0 (kR) \dfrac{I_z (k)}{(\alpha^2 + k^2)^{\frac{3}{2}}}
\end{equation}

where 
$$
\begin{equation}
\label{eq:integral}
I_z(k) = \dfrac{4}{\beta} \left\{ 1 - \dfrac{k}{k+\beta} \left[ e^{-z\beta} {}_2F_1\left(1, 1 + \frac{k}{\beta}; 2 + \frac{k}{\beta}; -e^{-z\beta} \right) + e^{z\beta} {}_2F_1\left(1, 1 + \frac{k}{\beta}; 2 + \frac{k}{\beta}; -e^{z\beta} \right)\right] \right\}
\end{equation}
$$

where $_2F_1$ is the Gaussian hypergeometric function, such that :

$$ _2F_1(a, b, c; z) = \sum_{n=0}^{\infty} \frac{(a)_n (b)_n}{(c)_n}\cdot \dfrac{z^n}{n!}$$ 

where $(a)_n$ is the Pochhammer symbol and $|z| < 1$.

This special function is implemented in scipy : https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.hyp2f1.html

**Note:**

I try here to simplify the Poisson equation the real space :
 
$$ \nabla^2 \Phi (R, z) = 4\pi G \rho_0 e^{\frac{-R}{R_d}} \cosh^{-2}{\frac{z}{z_d}}$$

which in cylindrical coordinates, with independence on the $\theta$ component reduces to:

$$ \dfrac{1}{R} \dfrac{\partial}{\partial R} \left( R \dfrac{\partial \Phi}{\partial R}\right) + \dfrac{\partial^2 \Phi}{\partial z^2} = 4\pi G \rho_0 e^{\frac{-R}{R_d}} \cosh^{-2}{\frac{z}{z_d}}$$

setting $z' = \dfrac{z}{z_d}$ and $R' = \dfrac{R}{R_d}$ , we can write simplify the equation in the following form :

$$ \dfrac{1}{R_{d}^{2}} \dfrac{1}{R'} \dfrac{\partial}{\partial R'} \left(R' \dfrac{\partial \Phi}{\partial R'}\right) + \dfrac{1}{z_{d}^{2}}\dfrac{\partial^2 \Phi}{\partial z'^2} = 4\pi G \rho_0 e^{-R'} \cosh^{-2}{z'}$$

Now, since $M_d = \rho_0 R_d \int_{0}^{2\pi} d\theta \int_{0}^{z_d} dz \int_{0}^{R_d} dr = 2\pi \rho_0 R_d^2 z_d$

We get :
$$ \dfrac{1}{R_{d}^{2}} \dfrac{1}{R'} \dfrac{\partial}{\partial R'} \left(R' \dfrac{\partial \Phi}{\partial R'}\right) + \dfrac{1}{z_{d}^{2}}\dfrac{\partial^2 \Phi}{\partial z'^2} = \frac{G M_d}{R_d^2 z_d}  e^{-R'} \cosh^{-2}{z'}$$

If we define $\phi'= \frac{\phi}{G M_d/z_d}$

$$ \dfrac{1}{R'} \dfrac{\partial}{\partial R'} \left(R' \dfrac{\partial \Phi'}{\partial R'}\right) + \dfrac{R_d^2}{z_{d}^{2}}\dfrac{\partial^2 \Phi'}{\partial z'^2} = e^{-R'} \cosh^{-2}{z'}$$

Usually one defines $\eta = \frac{z_d}{R_d} $

$$ \dfrac{1}{R'} \dfrac{\partial}{\partial R'} \left(R' \dfrac{\partial \Phi'}{\partial R'}\right) + \dfrac{1}{\eta^{2}}\dfrac{\partial^2 \Phi'}{\partial z'^2} = e^{-R'} \cosh^{-2}{z'}$$

Now the neural network should be function of 3 variables: $R'$, $z'$ and $\eta$.
Physically motivated values of $\eta$ can be around $0.1$ - $0.2$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as itg
from scipy.special import hyp2f1, jv

See [here](#Corrected-version) for the corrected version.

$R_d$ in the range $[2, 6]$ is ok

$z_d$, for now, we will set it in order to have $\dfrac{z_d}{R_d}=0.2$

The domain where to solve the equations can be between $\dfrac{R}{R_d} = [0;20]$  and $\dfrac{z}{z_d} = [0;5]$

We should also investigate where and when is possible to solve the equation via approximations.

To compare our results, we set here :
- $R_d = 4$  
- $\dfrac{z_d}{R_d}=0.2$  
- $M_d = 10^{10.5}$   

In [ ]:
Rd = 4
zd = 0.2 * Rd
alpha = 1 / Rd
beta = 2 / zd

In [ ]:
def potential_integral(k, R, z, alpha, beta):
    return jv(0, k * R) * Iz(z, k, b=beta) / (alpha**2 + k**2)**1.5

In [ ]:
def Iz(z, k, b):
    factor = k / (k + b)
    first_part = np.exp(-z * b) * hyp2f1(1, 1 + k / b, 2 + k / b, -np.exp(-z * b))
    second_part = np.exp(z * b) * hyp2f1(1, 1 + k / b, 2 + k / b, -np.exp(z * b))
    
    return (4 / b) * (1 - factor * (first_part + second_part))

We can integrate over a domain $R \times z$ using the `quad_vec` function : 

In [ ]:
Rtab = np.linspace(0, 20, 100) * Rd
ztab = np.linspace(0, 5, 100) * zd

Rv, zv = np.meshgrid(Rtab, ztab)

In [ ]:
vec_sol = itg.quad_vec(potential_integral, a=0, b=100, args=(Rv, zv, alpha, beta))

In [ ]:
plt.figure()
plt.pcolormesh(Rv, zv, vec_sol[0])
plt.xlabel("$R$")
plt.ylabel("$z$")
plt.colorbar(label="$\Phi(R, z)$");

We want to test with the data available. First column is $R$, second column is $z$ and the third column is the gravitational potential. 

In [ ]:
data = np.loadtxt("test_phi_grid.dat")
R_test, z_test, phi_test = data.T

R_test = R_test.reshape(250, 250)
z_test = z_test.reshape(250, 250)
phi_test = phi_test.reshape(250, 250)

In [ ]:
plt.figure()
plt.title('Pre-computed Potential')
plt.pcolormesh(R_test, z_test, phi_test)
plt.xlabel("$R$")
plt.ylabel("$z$")
plt.colorbar(label="$\Phi(R, z)$");

In [ ]:
%%time
phi = itg.quad_vec(potential_integral, a=0, b=200, args=(R_test, z_test, alpha, beta))

In [ ]:
print(f"Resultat: {0.00088:%}")

In [ ]:
Md = 10**(10.5)
G = 1 # 6.674 * 10 ** (-11)

In [ ]:
phi_ = - (G * Md)/(2 * zd * Rd**3) * phi[0]

In [ ]:
plt.figure()
plt.title('Potential computed with my code')
plt.pcolormesh(R_test, z_test, phi_)
plt.xlabel("$R$")
plt.ylabel("$z$")
plt.colorbar(label="$\Phi(R, z)$");

In [ ]:
plt.figure()
plt.title("Relative Error")
plt.pcolormesh(R_test, z_test, np.abs(phi_test - phi_)/phi_)
plt.xlabel("$R$")
plt.ylabel("$z$")
plt.colorbar(label="$\dfrac{\Delta \Phi(R, z)}{\Phi(R, z)}$");

In [ ]:
diff = np.abs(phi_test.flatten() - phi_.flatten())
plt.plot(diff[250:]);

In [ ]:
diff = np.abs(phi_test.flatten() - phi_.flatten())
plt.plot(diff[:250]);

In [ ]:
diff = np.abs((phi_test - phi_)/phi_).flatten()
plt.plot(diff[:250]);

In [ ]:
plt.plot(diff)
plt.yscale('log');

In [ ]:
counts, bins = np.histogram(np.abs(phi_test.flatten() - phi_.flatten()))
plt.bar(bins[:-1], counts, width=(bins[1]-bins[0]), log=True)
plt.xlabel("Absolute Error")
plt.ylabel("Count");

In [ ]:
counts, bins = np.histogram(np.abs((phi_test - phi_)/phi_).flatten())
plt.bar(bins[:-1], counts, width=(bins[1]-bins[0]), log=True)
plt.xlabel("Relative Error")
plt.ylabel("Count");

We wish to split the integration domain in two; $[0;\infty]$    becomes $[0,1] + [1,\infty]$

Then in the range $[1,\infty]$ we make a change of variable $x = \dfrac{1}{k}$ and try now to integrate that part between $1$ and $0$. 

We also want the following plots:

- 2D map of the relative error  
- histogram of the absolute and relative errors  
- zoom into the region of small radii (where the 2 solutions start to disagree?)

### Correction for larger values of $k$

It appears that we cannot integrate for values larger $k=\approx 250$ because of the part $${}_2F_1\left(1, 1 + \frac{k}{\beta}; 2 + \frac{k}{\beta}; -e^{z\beta} \right)$$

Indeed, the ${}_2F_1$ function is definded only for $|z| < 1$, but the factor $-e^{z'\beta}$ does not satisfy this condition. For small values of $k$ the fraction in the function stays balanced and returns small values, but for large $k$ the returned numbers get too small and `nan` values are returned.  We therefore need to find a way to extend the function for $|z| > 1$. Following https://arxiv.org/pdf/2010.08555.pdf we use :

$$\sum_{n=0}^{\infty} z^n + \sum_{n=0}^{\infty} z^n \sum_{l=1}^{\infty}(-1)^l \dfrac{n(1+n)^{l-1}}{y^l}$$

valid for $y \rightarrow +\infty$.

In [ ]:
k = np.linspace(0, 1000, 100)
b = beta
z = np.linspace(0, 4, 100)

Md = 10**(10.5)
G = 1 # 6.674 * 10 ** (-11)

In [ ]:
def approx_2F1(y, z, boundary=100_000):
    ntab = np.arange(0, boundary, 1).reshape(-1, 1)
    ltab = np.arange(1, boundary, 1).reshape(-1, 1)
    z = np.repeat(z.reshape(-1, 1), boundary, axis=1)
    z_sum = np.sum(z**ntab, axis=1)
    

In [ ]:
import numpy as np

def compute_sum(z, y, max_iter=100):
    """
    Compute the sum defined by the formula:
    sum_n=0^inf z^n + sum_n=0^inf z^n sum_l=1^inf (-1)^l * n * (1 + n)^(l - 1) / y^l

    :param z: numpy array, input values for the sum
    :param y: float, input value for the denominator of the second sum
    :param max_iter: int, maximum number of iterations for the second sum
    :return: numpy array, result of the sum for each input value in z
    """
    ntab = np.arange(0, max_iter, 1)
    z_sum1 = np.sum(z**ntab, axis=1)
    
    ntab = ntab.reshape((1, -1))
    ltab = np.arange(1, max_iter).reshape((1, -1))
    z = z.reshape((-1, 1))
    y = y.reshape((-1, 1))
    
    z_sum2 = np.sum((-1)**ltab * ntab * (1 + ntab)**(ltab - 1) / y**ltab * z**ntab.reshape((-1, 1)), axis=1)
    
    return z_sum1 + z_sum2

In [ ]:
compute_sum(-np.exp(z * b), k)

In [ ]:
import mpmath

In [ ]:
%%timeit
hyp2f1(1, 1 + k / b, 2 + k / b, -np.exp(z * b))

In [ ]:
%%timeit
mp_func = np.frompyfunc(mpmath.hyp2f1, 4, 1)
mp_func(1, 1 + k / b, 2 + k / b, -np.exp(z * b))

In [ ]:
%%timeit
mp_func = np.vectorize(mpmath.hyp2f1)
mp_func(1, 1 + k / b, 2 + k / b, -np.exp(z * b))

In [ ]:
mp_func = np.vectorize(mpmath.hyp2f1)

In [ ]:
%%timeit
mp_func(1, 1 + k / b, 2 + k / b, -np.exp(z * b))

In [ ]:
mp_func = np.frompyfunc(mpmath.hyp2f1, 4, 1)

In [ ]:
%%timeit
mp_func(1, 1 + k / b, 2 + k / b, -np.exp(z * b))

## Corrected version

Therefore we wish to rewrite the code using `mpmath` where needed:

In [ ]:
import mpmath

In [ ]:
Rd = 4
zd = 0.2 * Rd
alpha = 1 / Rd
beta = 2 / zd

In [ ]:
mp_func = np.frompyfunc(mpmath.hyp2f1, 4, 1)
def Iz(z, k, b):
    factor = k / (k + b)
    first_part = np.exp(-z * b) * hyp2f1(1, 1 + k / b, 2 + k / b, -np.exp(-z * b))
    second_part = np.exp(z * b) *  np.asarray(mp_func(1, 1 + k / b, 2 + k / b, -np.exp(z * b)), dtype=np.float64)
    
    return (4 / b) * (1 - factor * (first_part + second_part))

def potential_integral(k, R, z, alpha, beta):
    return jv(0, k * R) * Iz(z, k, b=beta) / (alpha**2 + k**2)**1.5


In [ ]:
data = np.loadtxt("test_phi_grid.dat")
R_test, z_test, phi_test = data.T

R_test = R_test.reshape(250, 250)
z_test = z_test.reshape(250, 250)
phi_test = phi_test.reshape(250, 250)

In [ ]:
%%time
phi = itg.quad_vec(potential_integral, a=0, b=np.inf, args=(R_test, z_test, alpha, beta))

The previous cell takes too much time to run, and is absolutely unusable.

In [ ]:
plt.figure()
plt.title("Relative Error")
plt.pcolormesh(R_test, z_test, np.abs(phi_test - phi_)/phi_)
plt.xlabel("$R$")
plt.ylabel("$z$")
plt.colorbar(label="$\dfrac{\Delta \Phi(R, z)}{\Phi(R, z)}$");

In [ ]:
counts, bins = np.histogram(np.abs((phi_test - phi_)/phi_).flatten())
plt.bar(bins[:-1], counts, width=(bins[1]-bins[0]), log=True)
plt.xlabel("Relative Error")
plt.ylabel("Count");